In [1]:
import networkx as nx
import numpy as np
import pickle
import time

In [6]:
def fullprint(*args, **kwargs):
    from pprint import pprint
    import numpy
    opt = numpy.get_printoptions()
    numpy.set_printoptions(threshold=numpy.inf)
    pprint(*args, **kwargs)
    numpy.set_printoptions(**opt)


class songGenerator:
    def __init__(self, filename):
        start_time = time.time()
        dataframe = pickle.load( open( filename, "rb" ) ) #get dataframe with scraped tabs from pickle file
        G = nx.MultiDiGraph() #initialize graph
        nodes = [] #will hold the tuple note that corresponds with each numbered node of the graph
        totalNotes = 0 #keeps count of how many notes are used in the graph
        #print('Making Graph')
        for songNum in range(0,dataframe.shape[0]): #iterate through all songs in the dataframe
           # if songNum % 1 == 0:
                #print('On song #',songNum)
            if dataframe.iloc[songNum,4] == 0: #currently will only use songs with no capo
                array = dataframe.iloc[songNum,5] #gets the TABS for the song
                totalNotes += array.shape[1] #increments number of notes based on length of this song
                if tuple(array[:,0]) not in G: #if the current note is not in the graph, add a corresponding node
                    G.add_node(tuple(array[:,0]))
                    nodes.append(tuple(array[:,0]))
                
                
                
                for i in range(1,array.shape[1]): #iterate through all notes in the song
                    current = tuple(array[:,i]) #turn each note into a tuple (because tuples are hashable and thus can be used as identifiers for the graph nodes)
                    prev = tuple(array[:,i-1])
                    check = time.time()
                    if current not in G: 
                        G.add_node(current)
                        nodes.append(current)
                    
                    G.add_edge(prev,current) #adds directed edge from the previous note to the current note
        print("Iterate Through Notes in Song in --- %s seconds ---" % (time.time() - start_time))
                #print(totalNotes)

        self.graph = G
        self.nodes = nodes
        self.adjacency_matrix = nx.to_numpy_array(G,nodelist=nodes)
        #print('Making Transition Matrix')
        self.transition_matrix = np.divide(self.adjacency_matrix ,np.array([np.sum(self.adjacency_matrix,axis=1)]).T) 
    
    def writeSongToFile(self,song,filename):
        songList = ['-'.join(row) for row in song.T]
        songList = [row + '\n' for row in songList]
        #print(songList)

        file1 = open(filename, 'w')
        file1.writelines(songList)
        file1.close()
        
    def makeSong(self,length):
        n = self.transition_matrix.shape[0]
        X = [np.random.choice(n, p=self.transition_matrix[0,:])]  # p is a needed input for np.random.choice()
        for t in range(1,length):
            X.append(np.random.choice(n, p=self.transition_matrix[X[t-1],:]))
        
        song = np.array([self.nodes[X[0]]])
        for i in range(1,len(X)):
            song = np.append(song,[self.nodes[X[i]]],axis=0)
        return song
    
    def makeSong(self,length,filename):
        #print('Making Song')
        n = self.transition_matrix.shape[0]
        X = [np.random.choice(n, p=self.transition_matrix[0,:])]  # p is a needed input for np.random.choice()
        for t in range(1,length):
            X.append(np.random.choice(n, p=self.transition_matrix[X[t-1],:]))
        
        song = np.array([self.nodes[X[0]]])
        for i in range(1,len(X)):
            song = np.append(song,[self.nodes[X[i]]],axis=0)
       # print('Writing Song to File')
        self.writeSongToFile(song,filename)
        
        return song
    
    
    
        
        

        

In [7]:
start_time = time.time()
generator = songGenerator('ScrapedSongsKeyC.p')
print("Total time --- %s seconds ---" % (time.time() - start_time))
song = generator.makeSong(100,'Test2.txt')


Iterate Through Notes in Song in --- 1.1035120487213135 seconds ---
Total time --- 3.167266845703125 seconds ---


<ipython-input-6-5949a264dc39>:46: RuntimeWarning: invalid value encountered in true_divide
  self.transition_matrix = np.divide(self.adjacency_matrix ,np.array([np.sum(self.adjacency_matrix,axis=1)]).T)


In [ ]:
test = pickle.load( open( "ScrapedSongsKeyC.p", "rb" ) )
#test = test = pickle.load( open( "ScrapedSongs100.p", "rb" ) )

array = test.iloc[0,5]
print(array[:,0:9],'\n')
#print(array.T)
print(test.shape)
test


In [ ]:
test_tuple = tuple(array[:,0])
G = nx.MultiDiGraph()
nodes = []
G.add_node(tuple(array[:,0]))
nodes.append(tuple(array[:,0]))

for i in range(1,array.shape[1]):
    current = tuple(array[:,i])
    prev = tuple(array[:,i-1])
    if current not in G: 
        G.add_node(current)
        nodes.append(current)
    G.add_edge(prev,current)
#nx.draw(G,with_labels=0,node_size=20,width=.1)
adjacency = nx.to_numpy_array(G,nodelist=nodes)
print(nodes,'\n')
print(adjacency.shape)


In [ ]:
start_time = time.time()
G = nx.MultiDiGraph()
nodes = []
totalNotes = 0
for songNum in range(0,867):
    #print('Song #',songNum)
    if test.iloc[songNum,4] == 0:
#         if songNum % 10 == 0:
#             print('On song number:',songNum)
        array = test.iloc[songNum,5]
        totalNotes += array.shape[1]
        if tuple(array[:,0]) not in G:
            G.add_node(tuple(array[:,0]))
            nodes.append(tuple(array[:,0]))
        
        #start_time = time.time()
        
        for i in range(1,array.shape[1]):
            current = tuple(array[:,i])
            prev = tuple(array[:,i-1])
            if current not in G: 
                G.add_node(current)
                nodes.append(current)
            G.add_edge(prev,current)
        #print("Iterate Through Notes in Song in --- %s seconds ---" % (time.time() - start_time))
        #print(totalNotes)
#nx.draw(G,with_labels=0,node_size = 5, width = .7)
adjacency = nx.to_numpy_array(G,nodelist=nodes)
adjacency.shape
#print(totalNotes)
print("Total time --- %s seconds ---" % (time.time() - start_time))



In [ ]:
import matplotlib.pyplot as plt

adjacency
#plt.imshow(adjacency)
np.savetxt('song.txt', song)

In [ ]:
# for row in adjacency:
#     if np.sum(row) == 0:
#         print("Error")

In [ ]:
#divide each element by the sum of its row to get transition probability matrix, in which each row sums to 1
transition_matrix = np.divide(adjacency ,np.array([np.sum(adjacency,axis=1)]).T) 
#fullprint(np.sum(transition_matrix,axis=1))


In [ ]:
n = transition_matrix.shape[0]
X = [np.random.choice(n, p=transition_matrix[0,:])]  # p is a needed input for np.random.choice()
for t in range(1,100):
    X.append(np.random.choice(n, p=transition_matrix[X[t-1],:])) 

In [ ]:
song = np.array([nodes[X[0]]])
for i in range(1,len(X)):
    song = np.append(song,[nodes[X[i]]],axis=0)
song[0:10,:].T


In [ ]:
print(nodes[0])
print(nodes[1])
#for(note in X):
np.append([nodes[0]],[nodes[1]],axis = 0).T

In [ ]:

def writeSongToFile(song,filename):
    file1 = open('songTest.txt', 'w')
    songList = [''.join(row) for row in song]
    

In [ ]:
# #Testing networkx graphs

# G = nx.MultiDiGraph()
# x = np.array([[1],[2],[3]])
# x = tuple(x[:,0])
# y = np.array([[4],[5],[6]])
# y = tuple(y[:,0])
# z = np.array([[7],[8],[9]])
# z = tuple(z[:,0])
# G.add_node(x)
# G.add_node(y)
# G.add_node(z)
# G.add_edge(x,y)
# G.add_edge(y,z)
# G.add_edge(x,z)
# G.add_edge(x,z)
# print(G.edges)
# nx.draw(G,with_labels=1)
# A = nx.to_numpy_array(G,nodelist=[x,y,z])
# A

In [ ]:
# #Testing on numpy arrays

# x = np.ones((3,3))
# z = np.divide(x,y.T)
# y = np.array([[1,2,3]])
# y.T
# z
# x += [[1],[2],[3]]
# print(x)
# print(np.array([np.sum(x,axis=1)]).T)
# np.divide(x ,np.array([np.sum(x,axis=1)]).T)
